# [Advent of Code 2018](https://adventofcode.com/2018)

A collection of common functions to be used across problems, will add more as a generic use case for each comes up!

In [1]:
import os
import re
from collections import Counter, defaultdict, namedtuple
from datetime import datetime
from itertools import cycle


cat = ''.join


Point = namedtuple('Point', 'x,y')


def Input(day):
    """Fetch the data input from disk."""
    filename = os.path.join('../data/advent2018/input{}.txt'.format(day))
    return open(filename)


def hamming_distance(s1, s2):
    """Number of non equal characters between two strings."""
    assert len(s1) == len(s2), 'Strings are not equal length'
    return sum(
        char1 != char2
        for char1, char2
        in zip(s1, s2)
    )


def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

## [Day 1: Chronal Calibration](https://adventofcode.com/2018/day/1)

The first part simply requires us to sum the values in a list to apply all the changes from the given delas.

In [2]:
def parse_input(initial_data):
    res = []
    for data in initial_data.readlines():
        res.append(int(data))
    return res

data = parse_input(Input(1))

In [3]:
sum(data)

525

Second portion requires us to find the first repeated value if we continually sum items in the delta list. A set is ideal here as it's O(1) for membership tests.

In [4]:
def find_first_repeat(deltas):
    seen = set([0])
    position = 0
    
    for delta in cycle(deltas):
        position += delta
        if position in seen:
            break
        seen.add(position)
    return position

assert find_first_repeat([7, 7, -2, -7, -4]) == 14

In [5]:
find_first_repeat(data)

75749

# [Day 2: Inventory Management System](https://adventofcode.com/2018/day/2)

In [6]:
data = [line.strip() for line in Input(2).readlines()]

In [7]:
from collections import Counter

def calculate_checksum(data):
    num_threes = 0
    num_twos = 0
    for line in data:
        counts = Counter(line)
        if 3 in counts.values():
            num_threes += 1
        if 2 in counts.values():
            num_twos += 1
        
    return num_threes * num_twos


test_data = [
    "abcdef",
    "bababc",
    "abbcde",
    "abcccd",
    "aabcdd",
    "abcdee",
    "ababab",
]
assert calculate_checksum(test_data) == 12
calculate_checksum(data)
    

6944

We're told that we need to find two strings that have only one character difference between them, this is also known as the the [Hamming Distance](https://en.wikipedia.org/wiki/Hamming_distance). So we're looking for two strings for which the hamming distance bewteen them is one, simple enough!

In [8]:
def find_boxes(data):
    for index, row in enumerate(data[:-1]):
        for comparison in data[index:]:
            if hamming_distance(row, comparison) == 1:
                same_chars = [
                    char1
                    for (char1, char2) in zip(row, comparison)
                    if char1 == char2
                ]
                return ''.join(same_chars)

find_boxes(data)

'srijafjzloguvlntqmphenbkd'

## [Day 3: No Matter How You Slice It](https://adventofcode.com/2018/day/3)

In [9]:
def parse_input(data):
    reg = re.compile('#(\d+) @ (\d+),(\d+): (\d+)x(\d+)')
    datum = namedtuple('datum', 'claim,left,top,width,height')
    parsed = []
    for line in data:
        match = reg.match(line)
        
        parsed.append(
            datum(*[int(i) for i in match.groups()])
        )
    return parsed


def find_overlaps(data):
    overlaps = defaultdict(list)
    repeated = 0
    
    for datum in data:
        for x in range(datum.width):
            for y in range(datum.height):
                p = Point(datum.left + x, datum.top + y)
                overlaps[p].append(datum.claim)
    return overlaps


def count_overlaps(data):
    overlaps = find_overlaps(data)
    return len([
        claims for claims in overlaps.values()
        if len(claims) > 1
    ])
    
        
test_data = [
    '#1 @ 1,3: 4x4',
    '#2 @ 3,1: 4x4',
    '#3 @ 5,5: 2x2',
]
test_data = parse_input(test_data)
assert count_overlaps(test_data) == 4

data = parse_input(Input(3).readlines())
count_overlaps(data)

110827

In [10]:
def find_unique(data):
    remainin_claims = set(
        datum.claim for datum in data
    )
    overlaps = find_overlaps(data)
    
    for claims in overlaps.values():
        if len(claims) == 1:
            continue
        remainin_claims -= set(claims)
    assert len(list(remainin_claims)) == 1
    return list(remainin_claims)[0]


assert find_unique(test_data) == 3
find_unique(data)

116

## [Day 4: Repose Record](https://adventofcode.com/2018/day/4)

The most fiddly portion of this problem was just parsing the data! The incoming events log are stateful, in that the current line applies to the most recently seen guard id.

In [11]:
Event = namedtuple('Event', 'asleep,wake_up')


def parse_data(lines):
    """
        Sort random order events before grouping them into
        (alseep, wake_up) datetime pairs.
    """
    data = []
    
    for line in lines:
        date_str = line[1:17]
        event = line[19:]
        data.append(
            (
                datetime.strptime(date_str, '%Y-%m-%d %H:%M'),
                event
            )
        )
    data = sorted(data)
    
    guard_events = defaultdict(list)

    current_guard = None
    data_iter = iter(data)
    try:
        while True:
            event = next(data_iter)
            guard_number = re.match('Guard #(\d+)', event[1])
            
            if guard_number:
                current_guard = int(guard_number.groups()[0])
                asleep = next(data_iter)
            else:
                asleep = event

            wakes_up = next(data_iter)

            guard_events[current_guard].append(
                Event(
                    asleep[0],
                    wakes_up[0]
                )
            )

    except StopIteration:
        pass        
    return guard_events

Phew, now that that's done we can get on with the problem!

All we need to do here is find the guard that's spent the most time asleep, the multiple the time he's most likely to be asleep (i.e the most observed sleepiest minute) and multiple that by the guard's id.

In [12]:
def minutes_asleep(guard_events):
    sleep_counter = defaultdict(int)
    
    for guard, events in guard_events.items():
        for asleep, wake_up in events:
            sleep_counter[guard] += wake_up.minute - asleep.minute
            
    return sleep_counter


def most_common_time_asleep(sleepiest_events):
    min_counter = Counter()
    for asleep, awake in sleepiest_events:
        min_counter.update(
            range(asleep.minute, awake.minute)
        )
    return min_counter.most_common(1)[0][0]

    
def strat_1(guard_events):
    sleep_counter = minutes_asleep(guard_events)

    sleepiest_guard = None
    max_time_asleep = 0
    for guard, time_asleep in sleep_counter.items():
        if time_asleep > max_time_asleep:
            max_time_asleep = time_asleep
            sleepiest_guard = guard
    
    common_minute = most_common_time_asleep(
        guard_events[sleepiest_guard]
    )
    return sleepiest_guard * common_minute

    

test_data = [
    '[1518-11-01 00:00] Guard #10 begins shift',
    '[1518-11-01 00:05] falls asleep',
    '[1518-11-01 00:25] wakes up',
    '[1518-11-01 00:30] falls asleep',
    '[1518-11-01 00:55] wakes up',
    '[1518-11-01 23:58] Guard #99 begins shift',
    '[1518-11-02 00:40] falls asleep',
    '[1518-11-02 00:50] wakes up',
    '[1518-11-03 00:05] Guard #10 begins shift',
    '[1518-11-03 00:24] falls asleep',
    '[1518-11-03 00:29] wakes up',
    '[1518-11-04 00:02] Guard #99 begins shift',
    '[1518-11-04 00:36] falls asleep',
    '[1518-11-04 00:46] wakes up',
    '[1518-11-05 00:03] Guard #99 begins shift',
    '[1518-11-05 00:45] falls asleep',
    '[1518-11-05 00:55] wakes up',
]

test_data = parse_data(test_data)
assert strat_1(test_data) == 240
data = parse_data(Input(4).readlines())
strat_1(data)

101262

In part two we're required to find the most common time to be asleep across all guards, and multuple that by the guard responsible for being asleep at that time.

In [13]:
def strat_2(guard_events):
    guard_counters = defaultdict(Counter)
    for guard, events in guard_events.items():
        for asleep, awake in events:
            guard_counters[guard].update(
                range(asleep.minute, awake.minute)
            )

    most_occurances = 0
    most_occurances_minute = 0
    most_occurances_guard = 0
    for guard, counter in guard_counters.items():
        most_common_min, occurances = counter.most_common(1)[0]
        if occurances > most_occurances:
            most_occurances = occurances
            most_occurances_minute = most_common_min
            most_occurances_guard = guard
            
    return most_occurances_guard * most_occurances_minute
            
    
assert strat_2(test_data) == 4455
strat_2(data)

71976

## [Day 5: Alchemical Reduction](https://adventofcode.com/2018/day/5)

This day specifically came with a warning about the size of the input data, so I downloaded it directly. This contained a new line which I did not strip, so got the wrong result for quite a while!

In [14]:
def reduce_polymer(data, remove_char=''):
    if remove_char:
        data = cat(char for char in data if char.lower() != remove_char.lower())
        
    data = list(data)
    index = 0
    
    while True:
        try:
            char = data[index]
            next_char = data[index + 1]
        except IndexError:
            break

        if char != next_char and char.lower() == next_char.lower():
            # Remove current char an the next
            del data[index]
            del data[index]
            if index > 0:
                index -= 1
        else:
            index += 1
    return cat(data)


test_input = 'dabAcCaCBAcCcaDA'
assert reduce_polymer(test_input) == 'dabCBAcaDA'

data = Input(5).read()
len(reduce_polymer(data))

9462

In [15]:
def ultimate_reduction(data):
    chars = set(data.lower())
    shortest = None
    for char in chars:
        reduced = reduce_polymer(data, char)
        if shortest is None or len(reduced) < shortest:
            shortest = len(reduced)
    return shortest


assert ultimate_reduction(test_input) == 4
ultimate_reduction(data)

4952